## Notebook can be used for cropping tiff images... works for 13 bands

based on https://github.com/omdena/wfp/blob/master/task4_resnet/13bands/data_preprocessing-allbands.ipynb

In [6]:
from PIL import Image
from matplotlib.pyplot import imshow
from pathlib import Path
from tqdm import tqdm
import numpy as np
from os import scandir, walk, listdir
from os.path import isfile, join
from skimage import io
import os
import shutil

In [7]:
LAUNCH = True

In [8]:
def open_allbands(path):
    return io.imread(str(path))

def center_crop_allbands(img, new_width=64, new_height=None):        

    
    height = img.shape[0]
    width = img.shape[1]

    if new_height is None:
        new_height = new_width

    left = int(np.ceil((width - new_width) / 2))
    right = width - int(np.floor((width - new_width) / 2))

    top = int(np.ceil((height - new_height) / 2))
    bottom = height - int(np.floor((height - new_height) / 2))
    
    #print(f'left {left}, right {right}, top {top}, bottom {bottom}')

   
    center_cropped_img = img[top:bottom,left:right,:]
    
    assert((center_cropped_img.shape[0] == new_height) and (center_cropped_img.shape[1] == new_width))

    return center_cropped_img

def crop_on_img_allbands(filename, path_target, size=64):
    img = open_allbands(filename)
    img = center_crop_allbands(img, size, size)
    print(path_target+filename.stem+'.tif')
    #img.save(path_target+filename.stem+'.tif')
    np.save(path_target+filename.stem+'.npy', img)
    return img

In [4]:
# crop_on_img_allbands(Path('/home/jupyter/data/WFP_Nepal_13Bands_Scale10/wheat_Oct16_85-80621341_27-5942076_20161030T044922_img_2.tif'), 'test_redim.npy')

In [2]:
%%time

# path_source = '/home/jupyter/data/WFP_Nepal_13Bands_Scale10/'
path_source = '/home/jupyter/data/kansas_13bands/'

# 64, 256, 500
size = 500

path_target = path_source + 'cropped_{}/'.format(size)

# get masks names in order to generate cropped files only for existing mask files.
mask_files = [f for f in listdir(path_source) if isfile(join(path_source, f))]

# recreate the dir for given cropped format
if os.path.exists(path_target):
    shutil.rmtree(path_target)
os.makedirs(path_target)

i = 0
y = 0

if LAUNCH:
    for r, d, f in walk(path_source):
        
        if r != path_source:
            break
        print(f'In directory {r}: {len(f)} files')
        for file in f:
            y = y + 1
            print(file)
            
            if file in mask_files:
                print(f'{i} working on img {path_source+file}')
                image = Path(path_source+file)    
                print(crop_on_img_allbands(image, path_target, size).shape)

                i = i + 1

print('processed {}, generated {} files'.format(y, i))